In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from util.load import DataLoader as Loader

loader = Loader(data_dir="../data")
trade_df = loader.load_consolidated_daily_data(
    ticker="kxhighny",
    max_days=100,
    type_="polysignal",
    verbose=True
)

trade_df.drop(columns=['time', 'trade_id', 'ticker']).head()

Loading kxhighny for 2025-03-25: 100%|██████████| 43/43 [00:00<00:00, 160.28it/s]


,time_to_strike,day_forecast_strike_dev,current_forecast_strike_dev,day_current_forecast_dev,day_wind_gusts_max,day_wind_speed_max,day_cloud_cover_max,day_cloud_cover_min,day_sunshine_duration,hour_wind_gusts,...,yes_price_sentiment_60,count_sentiment_60,yes_price_trend_60,count_agg_120,yes_price_vol_120,count_vol_120,taker_side_sentiment_120,yes_price_sentiment_120,count_sentiment_120,yes_price_trend_120
0,86378.686,1.3,-12.1,13.4,51.8,27.5,99,0,41428.27,18.4,...,43.333,100.733,-4,1511,2.637,99.877,1.0,43.333,100.733,-4
1,86378.686,1.3,-12.1,13.4,51.8,27.5,99,0,41428.27,18.4,...,43.333,100.733,-4,1511,2.637,99.877,1.0,43.333,100.733,-4
2,86378.686,1.3,-12.1,13.4,51.8,27.5,99,0,41428.27,18.4,...,43.333,100.733,-4,1511,2.637,99.877,1.0,43.333,100.733,-4
3,86378.686,1.3,-12.1,13.4,51.8,27.5,99,0,41428.27,18.4,...,43.333,100.733,-4,1511,2.637,99.877,1.0,43.333,100.733,-4
4,86378.686,1.3,-12.1,13.4,51.8,27.5,99,0,41428.27,18.4,...,43.333,100.733,-4,1511,2.637,99.877,1.0,43.333,100.733,-4


In [8]:
dist = [f for f in trade_df.columns.tolist() if f.startswith("strike_")]
trade_df[dist].tail()

,strike_m_5.0,strike_m_3.0,strike_m_1.0,strike_p_1.0,strike_p_3.0,strike_p_5.0
50266,4,4,3,6,92,3
50267,4,4,3,6,95,3
50268,4,4,3,6,96,3
50269,4,4,3,6,97,3
50270,4,4,3,6,97,3


In [6]:
class OnlineModel(nn.Module):

    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout=0.05, max_memory_size=100):
        super(OnlineModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(dropout),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(dropout),
            nn.LayerNorm(hidden_dim),
        )

        self.memory = nn.GRU(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        
        self.query = nn.Parameter(torch.randn(hidden_dim))
        self.key_transform = nn.Linear(hidden_dim, hidden_dim)
        self.value_transform = nn.Linear(hidden_dim, hidden_dim)

        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LeakyReLU(dropout),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, output_dim),
            nn.LayerNorm(output_dim),
            nn.Softmax(dim=-1),
        )

        self.hidden = None
        self.trade_memory = None
        self.max_memory_size = max_memory_size

    def _init_hidden(self):
        return torch.zeros(self.num_layers, self.hidden_dim)
    
    def _init_trade_memory(self):
        return torch.zeros(0, self.hidden_dim)
    
    def _attention(self, query, keys, values, batch_size: int = 1):
        query, keys, values = (
            query.unsqueeze(0), 
            keys.unsqueeze(0), 
            values.unsqueeze(0)
        )

        query = self.query.repeat(keys.size(0), 1)
        attn_scores = torch.bmm(
            self.key_transform(keys),
            query.unsqueeze(2)
        )
        attn_scores = attn_scores.squeeze(2)
        attn_weights = torch.softmax(attn_scores, dim=1).unsqueeze(1)
        context = torch.bmm(attn_weights, self.value_transform(values))
        return context.squeeze(1)
    
    def reset_state(self):
        self.hidden = None
        self.trade_memory = None

    def forward(self, x, return_state: bool = False):
        self.hidden = self._init_hidden() if self.hidden is None else self.hidden
        self.trade_memory = self._init_trade_memory() if self.trade_memory is None else self.trade_memory

        trade_encoded = self.encoder(x)
        output, self.hidden = self.memory(trade_encoded, self.hidden)

        if self.trade_memory.size(0) >= self.max_memory_size:
            self.trade_memory = self.trade_memory[1:, :]

        self.trade_memory = torch.cat([self.trade_memory, output], dim=0)

        context = self._attention(
            self.query,
            self.trade_memory,
            self.trade_memory
        )
        combined = torch.cat([context, trade_encoded], dim=1)
        output = self.output_layer(combined)
        if return_state:
            return output, self.trade_memory, self.hidden
        return output


In [4]:
# not f.split("_")[-1].isdigit() and
features = [
    f
    for f in trade_df.columns.tolist()
    if f not in ["time", "ticker", "trade_id"]
]

_, n_features = trade_df[features].shape
output_dim = 6
max_memory_size = 10
num_layers = 2
dropout = 0.05

model = OnlineModel(
    input_dim=n_features,
    hidden_dim=n_features,
    num_layers=num_layers,
    output_dim=output_dim,
    dropout=dropout,
    max_memory_size=max_memory_size
)

In [ ]:
for i in range(10):
    x = trade_df[features].iloc[i].values.reshape(1, -1)
    x = torch.from_numpy(x).float()
    model(x)

tensor([[[1.]]], grad_fn=<UnsqueezeBackward0>)
tensor([[[0.5314, 0.4686]]], grad_fn=<UnsqueezeBackward0>)
tensor([[[0.3810, 0.3359, 0.2831]]], grad_fn=<UnsqueezeBackward0>)
tensor([[[0.3138, 0.2767, 0.2332, 0.1764]]], grad_fn=<UnsqueezeBackward0>)
tensor([[[0.2601, 0.2293, 0.1933, 0.1462, 0.1712]]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[[0.2284, 0.2014, 0.1697, 0.1284, 0.1503, 0.1219]]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[[0.2058, 0.1814, 0.1529, 0.1157, 0.1355, 0.1098, 0.0989]]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[[0.1850, 0.1631, 0.1375, 0.1040, 0.1218, 0.0987, 0.0889, 0.1010]]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[[0.1676, 0.1478, 0.1245, 0.0942, 0.1103, 0.0895, 0.0806, 0.0915,
          0.0940]]], grad_fn=<UnsqueezeBackward0>)
tensor([[[0.1566, 0.1381, 0.1164, 0.0880, 0.1031, 0.0836, 0.0753, 0.0855,
          0.0878, 0.0655]]], grad_fn=<UnsqueezeBackward0>)
